# Summarizing and Aggregating

En este capítulo aprenderás sobre:

- El contexto de GroupBy y sus métodos disponibles, y cómo usarlos para analizar tus datos.
- Cómo agrupar datos basados en valores temporales usando los métodos `df.group_by_dynamic()`, `df.rolling()` y `Expr.over()`.
- Optimizaciones que puedes aplicar para mejorar el rendimiento.

---


## Split, Apply, and Combine

El enfoque "split, apply, combine" es fundamental para el análisis de datos agrupados. En Polars, este patrón se implementa de manera eficiente mediante el método `df.group_by()`. 

- **Split:** Separa el DataFrame en grupos según los valores de una o más columnas.
- **Apply:** Aplica funciones de agregación (como `sum()`, `mean()`, `count()`, etc.) a cada grupo.
- **Combine:** Une los resultados en un nuevo DataFrame, facilitando el análisis y la visualización.

Este método permite resumir grandes volúmenes de datos y extraer información relevante de manera rápida y reproducible. A continuación, veremos ejemplos prácticos de cómo utilizar `group_by` en Polars para realizar estas operaciones.

---

In [28]:
import polars as pl

## GroupBy Context

In [29]:
fruit = pl.read_csv("data/fruit.csv")
fruit_grouped = fruit.group_by("is_round")
fruit_grouped

In [30]:
fruit_grouped.len()

is_round,len
bool,u32
true,4
false,6


## Métodos disponibles en el contexto GroupBy

| Método                      | Descripción                                                                                   |
|-----------------------------|----------------------------------------------------------------------------------------------|
| GroupBy.__iter__()          | Permite iterar sobre los grupos (sub-DataFrames) generados por la operación `group_by`.      |
| GroupBy.agg(…)              | Calcula agregaciones para cada grupo (sub-DataFrame) de una operación `group_by`.            |
| GroupBy.all()               | Agrega los grupos en una Serie.                                                              |
| GroupBy.count()             | Devuelve el número de filas en cada grupo.                                                   |
| GroupBy.first()             | Agrega los primeros valores de cada grupo.                                                   |
| GroupBy.head(…)             | Obtiene las primeras n filas de cada grupo.                                                  |
| GroupBy.last()              | Agrega los últimos valores de cada grupo.                                                    |
| GroupBy.len(…)              | Devuelve el número de filas en cada grupo.                                                   |
| GroupBy.map_groups(…)       | Aplica una función personalizada (UDF) sobre los grupos como sub-DataFrame.                  |
| GroupBy.max()               | Reduce los grupos al valor máximo.                                                           |
| GroupBy.mean()              | Reduce los grupos al valor medio.                                                            |
| GroupBy.median()            | Devuelve la mediana por grupo.                                                               |
| GroupBy.min()               | Reduce los grupos al valor mínimo.                                                           |
| GroupBy.n_unique()          | Cuenta los valores únicos por grupo.                                                         |
| GroupBy.quantile(…)         | Calcula el cuantil por grupo.                                                                |
| GroupBy.sum()               | Reduce los grupos a la suma.                                                                 |
| GroupBy.tail(…)             | Obtiene las últimas n filas de cada grupo.                                                   |

- El método `df.set_sorted()` informa a Polars que la columna `positie` ya está ordenada. Esto permite que Polars aplique optimizaciones rápidas (fast path) que solo son posibles si sabe que los datos están ordenados.
- Estas optimizaciones mejoran el rendimiento al aprovechar el conocimiento sobre el orden de los datos.
- Es fundamental asegurarse de que la columna realmente esté ordenada antes de usar este método, ya que un uso incorrecto puede provocar resultados erróneos debido a suposiciones falsas sobre los datos.

In [31]:
top2000 = pl.read_excel(
    "data/top2000-2023.xlsx", read_options={"skip_rows": 1}
).set_sorted("positie")

In [32]:
top2000.head()

positie,titel,artiest,jaar
i64,str,str,i64
1,"""Bohemian Rhapsody""","""Queen""",1975
2,"""Roller Coaster""","""Danny Vera""",2019
3,"""Hotel California""","""Eagles""",1977
4,"""Piano Man""","""Billy Joel""",1974
5,"""Fix You""","""Coldplay""",2005


In [33]:
# Agrupa las canciones por año y concatena artista y título en una sola cadena por grupo.
(
    top2000.group_by("jaar")
    .agg(
        songs=pl.concat_str(             # Concatena strings
            pl.col("artiest"), pl.lit(" - "), pl.col("titel")
        ),
    )
    .sort("jaar", descending=True)
)

jaar,songs
i64,list[str]
2022,"[""Son Mieux - Multicolor"", ""Bankzitters - Je Blik Richting Mij"", … ""Måneskin - THE LONELIEST""]"
2021,"[""Goldband - Noodgeval"", ""Bankzitters - Stapelgek"", … ""Olivia Rodrigo - Drivers License""]"
2020,"[""DI-RECT - Soldier On"", ""Miss Montreal - Door De Wind"", … ""Dua Lipa ft. DaBaby - Levitating""]"
2019,"[""Danny Vera - Roller Coaster"", ""Floor Jansen & Henk Poort - Phantom Of The Opera"", … ""Tino Martin - Zij Weet Het""]"
2018,"[""Lady Gaga & Bradley Cooper - Shallow"", ""White Lies - Time To Give"", … ""Calvin Harris & Dua Lipa - One Kiss""]"
…,…
1960,"[""Etta James - At Last"", ""Shadows - Apache""]"
1959,"[""Jacques Brel - Ne Me Quitte Pas"", ""Elvis Presley - Hound Dog""]"
1958,"[""Chuck Berry - Johnny B. Goode"", ""Ella Fitzgerald & Louis Armstrong - Summertime""]"


In [34]:
(
    top2000.group_by("jaar", maintain_order=True) # Agrupar por año y mantener orden
    .head(3) # Tomar los top 3 de canciones por año
    .sort("jaar", descending=True) # Ajustar por año
    .head(9) # Tomar solo los 3 años más recientes su top3
)

jaar,positie,titel,artiest
i64,i64,str,str
2022,179,"""Multicolor""","""Son Mieux"""
2022,370,"""Je Blik Richting Mij""","""Bankzitters"""
2022,395,"""L'enfer""","""Stromae"""
2021,55,"""Noodgeval""","""Goldband"""
2021,149,"""Stapelgek""","""Bankzitters"""
2021,210,"""Dat Heb Jij Gedaan""","""Meau"""
2020,19,"""Soldier On""","""DI-RECT"""
2020,38,"""Door De Wind""","""Miss Montreal"""
2020,77,"""Impossible (Orchestral Version…","""Nothing But Thieves"""


In [35]:
(
    top2000.group_by("jaar", maintain_order=True)
    .tail(3)
    .sort("jaar", descending=True)
    .head(9)
 )

jaar,positie,titel,artiest
i64,i64,str,str
2022,1391,"""De Diepte""","""S10"""
2022,1688,"""Zeit""","""Rammstein"""
2022,1716,"""THE LONELIEST""","""Måneskin"""
2021,1865,"""Bon Gepakt""","""Donnie & Rene Froger"""
2021,1978,"""Hold On""","""Armin van Buuren ft. Davina Mi…"
2021,2000,"""Drivers License""","""Olivia Rodrigo"""
2020,1824,"""Smoorverliefd""","""Snelle"""
2020,1879,"""The Business""","""Tiësto"""
2020,1902,"""Levitating""","""Dua Lipa ft. DaBaby"""


In [36]:
(top2000.group_by("artiest").len().sort("len", descending=True).head(10))

artiest,len
str,u32
"""Queen""",34
"""The Beatles""",31
"""ABBA""",25
"""Bruce Springsteen""",22
"""The Rolling Stones""",22
"""Fleetwood Mac""",20
"""Coldplay""",20
"""Michael Jackson""",20
"""David Bowie""",18


In [37]:
sales = pl.read_csv("data/sales.csv")
sales.columns

['Date',
 'Day',
 'Month',
 'Year',
 'Customer_Age',
 'Age_Group',
 'Customer_Gender',
 'Country',
 'State',
 'Product_Category',
 'Sub_Category',
 'Product',
 'Order_Quantity',
 'Unit_Cost',
 'Unit_Price',
 'Profit',
 'Cost',
 'Revenue']

In [38]:
(
    sales.select("Product_Category", "Sub_Category", "Unit_Price")  
        .group_by("Product_Category", "Sub_Category")  
        .max()
        .sort("Unit_Price", descending=True)  
        .head(10)
)

Product_Category,Sub_Category,Unit_Price
str,str,i64
"""Bikes""","""Road Bikes""",3578
"""Bikes""","""Mountain Bikes""",3400
"""Clothing""","""Vests""",2384
"""Bikes""","""Touring Bikes""",2384
"""Accessories""","""Bike Stands""",159
"""Accessories""","""Bike Racks""",120
"""Clothing""","""Socks""",70
"""Clothing""","""Shorts""",70
"""Accessories""","""Hydration Packs""",55


In [39]:
(
    sales.select("Country", "Profit")
    .group_by("Country")
    .sum()
    .sort(by="Profit", descending=True)
)

Country,Profit
str,i64
"""United States""",11073644
"""Australia""",6776030
"""United Kingdom""",4413853
"""Canada""",3717296
"""Germany""",3359995
"""France""",2880282


In [40]:
(
    sales.select("Sub_Category", "Product")
    .group_by("Sub_Category")
    .n_unique()
    .sort("Product", descending=True)
    .head(10)
)

Sub_Category,Product
str,u32
"""Road Bikes""",38
"""Mountain Bikes""",28
"""Touring Bikes""",22
"""Tires and Tubes""",11
"""Jerseys""",8
"""Gloves""",4
"""Vests""",4
"""Bottles and Cages""",3
"""Shorts""",3


In [41]:
(
    sales.select("Age_Group", "Order_Quantity")
    .group_by("Age_Group")
    .mean()
    .sort("Order_Quantity", descending=True)
)

Age_Group,Order_Quantity
str,f64
"""Seniors (64+)""",13.530137
"""Youth (<25)""",12.124018
"""Adults (35-64)""",12.045303
"""Young Adults (25-34)""",11.560899


In [42]:
(
    sales.select("Age_Group", "Revenue")
    .group_by("Age_Group")
    .quantile(0.9)
    .sort("Revenue", descending=True)
)

Age_Group,Revenue
str,f64
"""Young Adults (25-34)""",2227.0
"""Adults (35-64)""",2217.0
"""Youth (<25)""",1997.0
"""Seniors (64+)""",943.0


## Advanced Methods

In [43]:
(
    sales.select("Country", "Profit", "Revenue")
    .group_by("Country")
    .agg(
        pl.col("Profit"),
        pl.col("Revenue"),
    )
)

Country,Profit,Revenue
str,list[i64],list[i64]
"""United Kingdom""","[1053, 1053, … 112]","[1728, 1728, … 184]"
"""Australia""","[1366, 1188, … 655]","[2401, 2088, … 1183]"
"""France""","[427, 427, … 655]","[787, 787, … 1207]"
"""Canada""","[590, 590, … 630]","[950, 950, … 1014]"
"""Germany""","[160, 53, … 746]","[295, 98, … 1250]"
"""United States""","[524, 407, … 542]","[929, 722, … 878]"


In [44]:
(
    sales.group_by("Country").agg(
        pl.col("Profit").alias("All Profits Per Transactions"),
        pl.col("Revenue").name.prefix("All "),
        Cost=pl.col("Revenue") - pl.col("Profit")
    )
)

Country,All Profits Per Transactions,All Revenue,Cost
str,list[i64],list[i64],list[i64]
"""Canada""","[590, 590, … 630]","[950, 950, … 1014]","[360, 360, … 384]"
"""United Kingdom""","[1053, 1053, … 112]","[1728, 1728, … 184]","[675, 675, … 72]"
"""Australia""","[1366, 1188, … 655]","[2401, 2088, … 1183]","[1035, 900, … 528]"
"""United States""","[524, 407, … 542]","[929, 722, … 878]","[405, 315, … 336]"
"""Germany""","[160, 53, … 746]","[295, 98, … 1250]","[135, 45, … 504]"
"""France""","[427, 427, … 655]","[787, 787, … 1207]","[360, 360, … 552]"


### Apply multiple aggregations at once

In [45]:
(    
    sales.select("Country", "Profit", "Revenue")
    .group_by("Country")
    .agg(
        pl.col("Profit").sum().name.prefix("Total "),
        pl.col("Profit").mean().alias("Average Profit per Transaction"),
        pl.col("Revenue").sum().name.prefix("Total "),
        pl.col("Revenue").mean().alias("Average Revenue per Transaction")
    )
)

Country,Total Profit,Average Profit per Transaction,Total Revenue,Average Revenue per Transaction
str,i64,f64,i64,f64
"""Australia""",6776030,283.089489,21302059,889.959016
"""Germany""",3359995,302.756803,8978596,809.028293
"""Canada""",3717296,262.187615,7935738,559.721964
"""France""",2880282,261.891435,8432872,766.764139
"""United States""",11073644,282.447687,27975547,713.552696
"""United Kingdom""",4413853,324.071439,10646196,781.659031


In [46]:
(
    sales.select("Country", "Profit", "Revenue")
    .group_by("Country")
    .agg(
        pl.all().sum().name.prefix("Total "),
        pl.all().mean().name.prefix("Average "),
    )
)

Country,Total Profit,Total Revenue,Average Profit,Average Revenue
str,i64,i64,f64,f64
"""France""",2880282,8432872,261.891435,766.764139
"""United States""",11073644,27975547,282.447687,713.552696
"""Germany""",3359995,8978596,302.756803,809.028293
"""Australia""",6776030,21302059,283.089489,889.959016
"""Canada""",3717296,7935738,262.187615,559.721964
"""United Kingdom""",4413853,10646196,324.071439,781.659031


In [47]:
(
    sales.select("Country", "Profit")
    .group_by("Country")
    .agg(
        (pl.col("Profit") > 1000).alias("Profit > 1000"),
        (pl.col("Profit") > 1000)
        .sum()
        .alias("Transactions with Profit > 1000"),
    )
)

Country,Profit > 1000,Transactions with Profit > 1000
str,list[bool],u32
"""Canada""","[false, false, … false]",868
"""Germany""","[false, false, … false]",659
"""United States""","[false, false, … false]",2623
"""France""","[false, false, … false]",482
"""Australia""","[true, true, … false]",1233
"""United Kingdom""","[true, true, … false]",788


 The *meta.root_names()* method returns the names of the columns from which
 the expression originates.

In [48]:
def sum_transactions_above_threshold(
    col: pl.Expr, threshold: float
) -> tuple[pl.Expr, pl.Expr]:
    """Sums transactions where the column col exceeds specified threshold"""
    original_column_name = col.meta.root_names()[0]
    condition_column = (col > threshold).alias(
        f"{original_column_name} > {threshold}"
    )
    new_column = (
        (col > threshold)
        .sum()
        .alias(f"Transactions with {original_column_name} > {threshold}")
    )
    return condition_column, new_column

sales.select("Country", "Profit").group_by("Country").agg(
    *sum_transactions_above_threshold(pl.col("Profit"), 1500)
)

Country,Profit > 1500,Transactions with Profit > 1500
str,list[bool],u32
"""United States""","[false, false, … false]",955
"""France""","[false, false, … false]",270
"""Australia""","[false, false, … false]",567
"""United Kingdom""","[false, false, … false]",457
"""Canada""","[false, false, … false]",303
"""Germany""","[false, false, … false]",317


## Row-Wise Aggregations

Polars ofrece varias funciones para realizar agregaciones horizontales (row-wise) de manera eficiente y vectorizada. Las más flexibles son `pl.reduce()` y `pl.fold()`, que permiten aplicar funciones personalizadas sobre varias columnas de cada fila.

**Resumen de métodos:**

| Método         | Descripción                                                                                   |
|----------------|----------------------------------------------------------------------------------------------|
| `pl.reduce()`  | Aplica una función acumuladora sobre una lista de expresiones, usando el primer valor como acumulador inicial. |
| `pl.fold()`    | Similar a `reduce`, pero permite especificar un valor inicial para el acumulador (`acc`).     |

**Argumentos principales (`pl.reduce` y `pl.fold`):**

| Argumento | Descripción                                                                                       |
|-----------|--------------------------------------------------------------------------------------------------|
| `function`| Función que recibe el acumulador y el valor actual, y devuelve el nuevo valor del acumulador.     |
| `exprs`   | Lista de expresiones (columnas) sobre las que se aplica la función.                              |
| `acc`     | (Solo en `fold`) Valor inicial del acumulador.                                                    |

Esto crea una nueva columna `"weight_plus_name_length"` que suma el peso y la longitud del nombre para cada fila. Puedes adaptar la función y las columnas según la lógica de agregación que necesites.

In [49]:
fold_example = pl.DataFrame({"col1": [2], "col2": [3], "col3": [4]})

fold_example.with_columns(
    sum=pl.fold(
        acc=pl.lit(0),
        function=lambda acc, x: acc + x,
        exprs=pl.col("*"),
    )
)

col1,col2,col3,sum
i64,i64,i64,i64
2,3,4,9


In [50]:
# Define the product sales DataFrame
products = pl.DataFrame(
    {
        "product_A": [10, 20, 30],
        "product_B": [20, 30, 40],
        "product_C": [30, 40, 50],
    }
)

# Define weights for each product
weights = {"product_A": 0.2, "product_B": 0.3, "product_C": 0.5}

# Create Polars expressions that multiply each column by its respective weight
weights_exprs = [
    (pl.col(product) * weight).alias(product)
    for product, weight in weights.items()
]

# Apply the fold function to calculate the weighted sum row-wise
# - Start with an initial value of 0 for the accumulator
# - Use a summing function (lambda acc, x: acc + x)
# - Pass the weighted expressions to the fold function
products_with_weighted_sum = products.with_columns(
    weighted_sum=pl.fold(
        acc=pl.lit(0),
        function=lambda acc, x: acc + x,
        exprs=weights_exprs,
    )
)

products_with_weighted_sum

product_A,product_B,product_C,weighted_sum
i64,i64,i64,f64
10,20,30,23.0
20,30,40,33.0
30,40,50,43.0


## Window Functions in Selection Context

Las funciones de ventana permiten agregar información contextual a cada fila sin reducir el DataFrame a grupos agregados. El método `Expr.over()` en Polars permite realizar agregaciones sobre grupos definidos por columnas, manteniendo la estructura original del DataFrame y enriqueciendo cada fila con información del grupo.

**Ventajas principales:**
- Permite agregar estadísticas de grupo (ej. suma, media) como nuevas columnas.
- Mantiene el número de filas original.
- Útil para análisis donde se requiere el contexto de cada fila junto con métricas agregadas del grupo.

---

### Argumentos de `Expr.over()`

| Argumento         | Descripción                                                                                                                                                                                                                   |
|-------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `partition_by`    | Columnas por las que se agrupa. Acepta expresiones o nombres de columna como string.                                                                                                                                        |
| `*more_exprs`     | Columnas adicionales para agrupar, especificadas como argumentos posicionales.                                                                                                                                              |
| `order_by`        | Ordena las funciones de ventana/agregaciones dentro de los grupos según la expresión pasada.                                                                                                                                |
| `mapping_strategy`| Estrategia para mapear los resultados de la agregación al DataFrame original:<br>• `group_to_rows`: Asigna los resultados a su posición original si el tamaño del grupo no cambia.<br>• `join`: Une los grupos como listas.<br>• `explode`: Explota los datos agrupados en nuevas filas, cambiando el número de filas. |

---

**Nota:**  
- `group_to_rows` mantiene la correspondencia fila-grupo si el tamaño no cambia.  
- `join` puede ser intensivo en memoria.  
- `explode` modifica el número de filas y requiere que los grupos estén ordenados para que el resultado tenga sentido.

In [51]:
(
    top2000.select(
        "jaar",
        "artiest",
        "titel",
        "positie",
        year_rank=pl.col("positie").rank().over("jaar"),
    )
    .sample(10, seed=42)
)

jaar,artiest,titel,positie,year_rank
i64,str,str,i64,f64
1985,"""Paul McCartney & Frog Chorus""","""We All Stand Together""",1622,45.0
1977,"""ABBA""","""Take A Chance On Me""",636,23.0
1970,"""Carpenters""","""(They Long To Be) Close To You""",1961,33.0
2021,"""Taylor Swift""","""Willow""",1399,38.0
1985,"""Chris Rea""","""Josephine""",1584,42.0
1999,"""Gé Reinders""","""Blaosmuziek""",1174,14.0
1990,"""Frank Boeijen Groep""","""Zeg Me Dat Het Niet Zo Is""",251,5.0
2008,"""Metallica""","""The Unforgiven III""",1210,11.0
1991,"""Chris Isaak""","""Wicked Game""",416,19.0


## Agrupación Dinámica

Cuando trabajas con datos temporales, puede ser útil crear grupos basados en una ventana de tiempo. Aquí es donde entra el método `df.group_by_dynamic()`. Este método calcula una ventana temporal de tamaño y longitud fijos, a la que asigna las filas del DataFrame. Es diferente de un `df.group_by()` normal, porque las filas pueden aparecer en múltiples ventanas de tiempo, dependiendo del tamaño y la longitud de los pasos de la ventana. Esto es útil para calcular datos de ventas anuales o trimestrales, donde quieres dividir los datos en periodos específicos.

### Tabla 1. Argumentos del método `df.group_by_dynamic()`

| Argumento | Descripción |
|-----------|-------------|
| every     | El intervalo en el que comienzan las ventanas. |
| offset    | Desplaza el inicio de la ventana. Por ejemplo, para iniciar la ventana a las 9 a.m. cada día y alinearla con el horario laboral, puedes usar `every="1d"` y `offset="9h"`. |
| period    | La duración de la ventana de tiempo. Coincide con `every` si no se especifica, resultando en grupos adyacentes y no superpuestos. Si quieres ventanas superpuestas, puedes establecer `period` a un valor mayor que `every`. |
| start_by  | Estrategia para determinar el inicio de la primera ventana, permitiendo alinearla con el primer dato, con un día específico de la semana, o ajustando al primer timestamp y aplicando el offset según el intervalo `every`. |

Los argumentos `every`, `period` y `offset` se pueden especificar usando las siguientes cadenas de duración:

### Tabla 2. Cadenas de duración y su significado

| Cadena   | Descripción           |
|----------|-----------------------|
| 1ns      | 1 nanosegundo         |
| 1us      | 1 microsegundo        |
| 1ms      | 1 milisegundo         |
| 1s       | 1 segundo             |
| 1m       | 1 minuto              |
| 1h       | 1 hora                |
| 1d       | 1 día calendario      |
| 1w       | 1 semana calendario   |
| 1mo      | 1 mes calendario      |
| 1q       | 1 trimestre calendario|
| 1y       | 1 año calendario      |
| 1i       | 1 conteo de índice    |

Estas cadenas se pueden combinar, por ejemplo: `"1y6m1w5d"` sería 1 año, 6 meses, 1 semana y 5 días.

Adicionalmente, el argumento `closed` determina si los valores que son exactamente el límite inferior o superior se incluyen o excluyen.

### Tabla 3. Valores posibles para el argumento `closed` en `df.group_by_dynamic()`

| Argumento | Descripción                                               | Intervalo | Incluye a | Incluye b |
|-----------|-----------------------------------------------------------|-----------|-----------|-----------|
| left      | El límite inferior es inclusivo, el superior es exclusivo | [a, b)    | ✓         | ✗         |
| right     | El límite inferior es exclusivo, el superior es inclusivo | (a, b]    | ✗         | ✓         |
| both      | Ambos límites son inclusivos                              | [a, b]    | ✓         | ✓         |
| none      | Ambos límites son exclusivos                              | (a, b)    | ✗         | ✗         |

### Agregaciones Rolling (Ventana Móvil)

Las agregaciones rolling permiten calcular métricas como medias móviles o sumas acumuladas sobre ventanas que se ajustan dinámicamente alrededor de los valores de una columna (por ejemplo, fechas). El método `df.rolling()` de Polars crea una ventana para cada valor de la columna de referencia, extendiéndose hacia atrás por el periodo especificado. Es útil para analizar tendencias temporales, como el promedio móvil de ventas por tienda.

**Principales argumentos de `df.rolling()`:**

| Argumento      | Descripción                                                                                           |
|----------------|------------------------------------------------------------------------------------------------------|
| index_column   | Columna que sirve como punto de referencia para la ventana (ej. timestamps).                         |
| period         | Tamaño de la ventana (ej. "7d" para 7 días).                                                         |
| offset         | Desplaza la ventana hacia adelante o atrás.                                                          |
| closed         | Define cómo se incluyen los valores en los límites de la ventana (igual que en `group_by_dynamic`).  |
| group_by       | Permite aplicar la agregación rolling dentro de grupos definidos por una o más columnas.              |

Estas opciones permiten calcular métricas móviles ajustadas por tiempo y por grupo, facilitando el análisis de tendencias y variabilidad en series temporales.

In [52]:
# Genera un rango de fechas cada 2 días entre el 1 y el 26 de abril de 2024
dates = pl.date_range(
    start=pl.date(2024, 4, 1),
    end=pl.date(2024, 4, 26),
    interval="2d",
    eager=True,
)
# Filtra solo días de lunes a sábado (excluye domingos)
dates = dates.filter(dates.dt.weekday() < 6)

# Duplica las fechas para simular dos tiendas y las ordena
dates_repeated = pl.concat([dates, dates]).sort()

# Crea un DataFrame pequeño de ventas por tienda y fecha
small_sales_df = (
    pl.DataFrame(
        {
            "date": dates_repeated,
            "store": ["Store A", "Store B"] * dates.len(),  # Alterna tiendas
            "sales": [
                200, 150, 220, 160, 250, 180, 270, 190, 280, 210,
                210, 170, 220, 180, 240, 190, 250, 200, 260, 210,
            ],
        }
    )
    .set_sorted("date")   # Informa a Polars que la columna 'date' está ordenada
    .set_sorted("store")  # Informa a Polars que la columna 'store' está ordenada
)
small_sales_df

date,store,sales
date,str,i64
2024-04-01,"""Store A""",200
2024-04-01,"""Store B""",150
2024-04-03,"""Store A""",220
2024-04-03,"""Store B""",160
2024-04-05,"""Store A""",250
…,…,…
2024-04-19,"""Store B""",190
2024-04-23,"""Store A""",250
2024-04-23,"""Store B""",200


In [53]:
result = small_sales_df.rolling(
    index_column="date",
    period="7d",
    group_by="store",
).agg(
    sum_of_last_7_days=pl.sum("sales"),
)

final_df = small_sales_df.join(
    result,
    on=["date", "store"],
)
final_df

date,store,sales,sum_of_last_7_days
date,str,i64,i64
2024-04-01,"""Store A""",200,200
2024-04-03,"""Store A""",220,420
2024-04-05,"""Store A""",250,670
2024-04-09,"""Store A""",270,740
2024-04-11,"""Store A""",280,800
…,…,…,…
2024-04-15,"""Store B""",170,570
2024-04-17,"""Store B""",180,560
2024-04-19,"""Store B""",190,540


## Upsampling con Polars

El método `df.upsample()` permite aumentar la resolución temporal de tus datos, generando nuevas filas en intervalos regulares donde no existen observaciones. Es útil cuando necesitas rellenar huecos en series temporales para análisis o visualización.

### Argumentos principales de `df.upsample()`

| Argumento        | Descripción                                                                                                 |
|------------------|------------------------------------------------------------------------------------------------------------|
| `time_column`    | Columna temporal usada para determinar el rango de fechas. Debe estar ordenada para que el resultado tenga sentido. |
| `every`          | Intervalo de duración para crear nuevas entradas (ejemplo: `"1d"` para cada día, `"1h"` para cada hora).   |
| `group_by`       | (Opcional) Agrupa por estas columnas antes de aplicar el upsampling en cada grupo.                         |
| `maintain_order` | (Opcional) Mantiene el orden predecible en la salida. Puede ser más lento si se activa.                    |


Este método es el opuesto a las agregaciones temporales, ya que en vez de resumir, expande la serie temporal para rellenar los intervalos faltantes.

In [57]:
upsampled_small_sales_df = small_sales_df.upsample(
    time_column="date",
    every="1d",
    group_by="store",
    maintain_order=True,
)
upsampled_small_sales_df

date,store,sales
date,str,i64
2024-04-01,"""Store A""",200
2024-04-01,"""Store B""",150
2024-04-03,"""Store A""",220
2024-04-03,"""Store B""",160
2024-04-05,"""Store A""",250
…,…,…
2024-04-19,"""Store B""",190
2024-04-23,"""Store A""",250
2024-04-23,"""Store B""",200


In [58]:
upsampled_small_sales_df.select(
    "date", pl.col("store").forward_fill(), pl.col("sales").interpolate()
)

date,store,sales
date,str,f64
2024-04-01,"""Store A""",200.0
2024-04-01,"""Store B""",150.0
2024-04-03,"""Store A""",220.0
2024-04-03,"""Store B""",160.0
2024-04-05,"""Store A""",250.0
…,…,…
2024-04-19,"""Store B""",190.0
2024-04-23,"""Store A""",250.0
2024-04-23,"""Store B""",200.0


## Takeaways

En este capítulo aprendiste a realizar agregaciones sobre tus datos utilizando Polars:

- Las agregaciones básicas disponibles en el contexto GroupBy, como `sum()`, `mean()`, `quantile()`, y `median()`.
- Las agregaciones avanzadas con `GroupBy.agg()`, que permiten combinar múltiples funciones, controlar nombres de columnas y agregar elementos como listas por grupo.
- Cómo realizar agregaciones sobre grupos en el contexto de selección usando el método `Expr.over()`.
- Cómo crear grupos basados en ventanas temporales con `df.group_by_dynamic()`.
- Cómo calcular agregaciones móviles (rolling) alrededor de los valores de tu DataFrame usando el método `df.rolling()`.